# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
# identify file path
weather_csv = '../main_script/weather_data.csv'

# turn csv into dataframe
weather_df = pd.read_csv(weather_csv)

weather_df.head()


,Name,Country,Latitude,Longitude,Temperature(F),Wind Speed(mph),Cloud Coverage,Humidity
0,Carnarvon,AU,-24.8667,113.6333,84.470,5.66,0,70
1,Bathsheba,BB,13.2167,-59.5167,79.070,9.77,20,69
2,Hermanus,ZA,-34.4187,19.2345,63.266,0.89,3,84
3,Mataura,NZ,-46.1927,168.8643,53.276,8.49,56,76
4,Cape Town,ZA,-33.9258,18.4232,66.272,1.54,75,88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

# convert humidity to float, drop NaN values
weather_df = weather_df.dropna()
humidity = weather_df['Humidity'].astype(float)

# store lattitude and longitude together
locations = weather_df[['Latitude', 'Longitude']].astype(float)

# create humidity heatmap layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, 
                                     weights = humidity, 
                                     dissipating = False, 
                                     max_intensity = max(weather_df['Humidity']), 
                                     point_radius = 2)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# copy weather_df
ideal_weather_df = weather_df.copy()
ideal_weather_df = ideal_weather_df.dropna()

# filter through df to find cities with ideal conditions
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Temperature(F)'] < 70) & (ideal_weather_df['Temperature(F)'] > 60)]
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Wind Speed(mph)'] < 10)]
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df['Cloud Coverage'] < 60) & (ideal_weather_df['Cloud Coverage'] > 10)]

ideal_weather_df


,Name,Country,Latitude,Longitude,Temperature(F),Wind Speed(mph),Cloud Coverage,Humidity
27,Chui,UY,-33.6971,-53.4616,65.318,1.25,21,83
66,Pleasant Point,NZ,-44.2667,171.1333,65.264,3.13,30,34
135,Los Llanos de Aridane,ES,28.6585,-17.9182,64.670,7.72,20,63
159,Cabo San Lucas,MX,22.8909,-109.9124,68.270,1.03,20,73
187,Arona,ES,28.0996,-16.6810,63.266,4.12,20,67
191,Praia da Vitória,PT,38.7333,-27.0667,62.276,5.18,40,94
200,Corpus Christi,US,27.8006,-97.3964,61.268,4.12,20,72
274,Katakwi,UG,1.8911,33.9661,66.776,0.51,11,83
331,Departamento de Maldonado,UY,-34.6667,-54.9167,66.470,1.54,20,88
390,Agüimes,ES,27.9054,-15.4461,63.266,8.23,40,77


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
row_index = 0

# limit hotel search within 5000m of each location
params = {'radius': 5000, 
          'types': 'lodging',
          'key': g_key}

# walk through hotel_df
for i, row in hotel_df.iterrows():
        
    # grab lat and lng data
    lat = row['Latitude']
    lng = row['Longitude']
    
    # add lat and lng data to params dict
    params['location'] = f'{lat}, {lng}'
    
    # API request
    print(f'Request number {row_index} for {row["Name"]}.')
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    
    try:
        print(f"Closest hotel: {results[0]['name']}")
        hotel_df.loc[row_index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print(' ')
    
    row_index = row_index + 1

Request number 0 for Chui.
Closest hotel: Nuevo Hotel Plaza
 
Request number 1 for Pleasant Point.
Closest hotel: Pleasant Point Hotel
 
Request number 2 for Los Llanos de Aridane.
Closest hotel: Valle Aridane
 
Request number 3 for Cabo San Lucas.
Closest hotel: Hotel Tesoro Los Cabos
 
Request number 4 for Arona.
Closest hotel: Paradise Court
 
Request number 5 for Praia da Vitória.
Closest hotel: Hotel Branco I
 
Request number 6 for Corpus Christi.
Closest hotel: Holiday Inn Corpus Christi Downtown Marina
 
Request number 7 for Katakwi.
Closest hotel: Ebenezer Guest House
 
Request number 8 for Departamento de Maldonado.
Closest hotel: La Cimarrona
 
Request number 9 for Agüimes.
Closest hotel: Hotel Villa de Aguimes
 
Request number 10 for Taltal.
Closest hotel: Residencial Karime
 
Request number 11 for Lagoa.
Closest hotel: Casa Das Faias
 
Request number 12 for Port Said.
Closest hotel: Aracan Hotel
 
Request number 13 for Adeje.
Closest hotel: Gran Tacande Wellness & Relax
 


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

locations_df = hotel_df[["Latitude", "Longitude"]]

locations_df = locations_df.dropna()
locations_df.drop(locations_df.loc[locations_df['Longitude'] < -90].index, inplace=True)
locations_df.drop(locations_df.loc[locations_df['Longitude'] > 90].index, inplace=True)
print(locations_df)


     Latitude  Longitude
27   -33.6971   -53.4616
135   28.6585   -17.9182
187   28.0996   -16.6810
191   38.7333   -27.0667
274    1.8911    33.9661
331  -34.6667   -54.9167
390   27.9054   -15.4461
400  -25.4000   -70.4833
431   39.0500   -27.9833
442   31.2565    32.2841
571   28.1227   -16.7260


In [20]:
# Add marker layer ontop of heat map and display
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

InvalidPointException: nan is not a valid latitude. Latitudes must lie between -90 and 90.